In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').getOrCreate()

21/08/22 13:48:54 WARN Utils: Your hostname, laha-ubuntu resolves to a loopback address: 127.0.1.1; using 10.4.130.205 instead (on interface enp6s0)
21/08/22 13:48:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/22 13:48:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
col_names = ['time_stamp','id', 'dlc','d0','d1','d2','d3','d4','d5','d6','d7','R']

In [4]:
fuzzy_data = spark.read.csv(
    "data/Fuzzy_dataset.csv", 
    header=None
)

In [5]:
fuzzy_data.show(n=5)

+-----------------+----+---+---+---+---+---+---+---+----+----+----+
|              _c0| _c1|_c2|_c3|_c4|_c5|_c6|_c7|_c8| _c9|_c10|_c11|
+-----------------+----+---+---+---+---+---+---+---+----+----+----+
|1478195721.903877|0545|  8| d8| 00| 00| 8a| 00| 00|  00|  00|   R|
|1478195721.905736|02b0|  5| ff| 7f| 00| 05| 49|  R|null|null|null|
|1478195721.908437|0002|  8| 00| 00| 00| 00| 00| 01|  07|  15|   R|
|1478195721.908676|0153|  8| 00| 21| 10| ff| 00| ff|  00|  00|   R|
|1478195721.909414|0130|  8| 19| 80| 00| ff| fe| 7f|  07|  60|   R|
+-----------------+----+---+---+---+---+---+---+---+----+----+----+
only showing top 5 rows



In [6]:
fuzzy_data.count()

3838860

In [7]:
fuzzy_data=fuzzy_data.na.drop()
fuzzy_data.count()

3751024

In [8]:
from functools import reduce
    
def rename_col(data, col_names=col_names):
    oldColumns = data.schema.names
    data = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], col_names[idx]), 
                        range(len(oldColumns)), data)
    return data

In [9]:
fuzzy_data = rename_col(fuzzy_data)
fuzzy_data.show(n=5)

+-----------------+----+---+---+---+---+---+---+---+---+---+---+
|       time_stamp|  id|dlc| d0| d1| d2| d3| d4| d5| d6| d7|  R|
+-----------------+----+---+---+---+---+---+---+---+---+---+---+
|1478195721.903877|0545|  8| d8| 00| 00| 8a| 00| 00| 00| 00|  R|
|1478195721.908437|0002|  8| 00| 00| 00| 00| 00| 01| 07| 15|  R|
|1478195721.908676|0153|  8| 00| 21| 10| ff| 00| ff| 00| 00|  R|
|1478195721.909414|0130|  8| 19| 80| 00| ff| fe| 7f| 07| 60|  R|
|1478195721.909660|0131|  8| 17| 80| 00| 00| 65| 7f| 07| 9f|  R|
+-----------------+----+---+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [10]:
columns_to_drop = ['time_stamp','R']

fuzzy_data = fuzzy_data.drop(*columns_to_drop)
fuzzy_data.show(n=5)

+----+---+---+---+---+---+---+---+---+---+
|  id|dlc| d0| d1| d2| d3| d4| d5| d6| d7|
+----+---+---+---+---+---+---+---+---+---+
|0545|  8| d8| 00| 00| 8a| 00| 00| 00| 00|
|0002|  8| 00| 00| 00| 00| 00| 01| 07| 15|
|0153|  8| 00| 21| 10| ff| 00| ff| 00| 00|
|0130|  8| 19| 80| 00| ff| fe| 7f| 07| 60|
|0131|  8| 17| 80| 00| 00| 65| 7f| 07| 9f|
+----+---+---+---+---+---+---+---+---+---+
only showing top 5 rows



In [11]:
fuzzy_data.rdd.zipWithIndex().take(5)

[(Row(id='0545', dlc='8', d0='d8', d1='00', d2='00', d3='8a', d4='00', d5='00', d6='00', d7='00'),
  0),
 (Row(id='0002', dlc='8', d0='00', d1='00', d2='00', d3='00', d4='00', d5='01', d6='07', d7='15'),
  1),
 (Row(id='0153', dlc='8', d0='00', d1='21', d2='10', d3='ff', d4='00', d5='ff', d6='00', d7='00'),
  2),
 (Row(id='0130', dlc='8', d0='19', d1='80', d2='00', d3='ff', d4='fe', d5='7f', d6='07', d7='60'),
  3),
 (Row(id='0131', dlc='8', d0='17', d1='80', d2='00', d3='00', d4='65', d5='7f', d6='07', d7='9f'),
  4)]

In [12]:
header = ['index'] + fuzzy_data.columns
fuzzy_data = fuzzy_data.rdd.zipWithIndex().map(lambda x: [x[1]] + list(x[0])).toDF(header)
fuzzy_data.show(5)

+-----+----+---+---+---+---+---+---+---+---+---+
|index|  id|dlc| d0| d1| d2| d3| d4| d5| d6| d7|
+-----+----+---+---+---+---+---+---+---+---+---+
|    0|0545|  8| d8| 00| 00| 8a| 00| 00| 00| 00|
|    1|0002|  8| 00| 00| 00| 00| 00| 01| 07| 15|
|    2|0153|  8| 00| 21| 10| ff| 00| ff| 00| 00|
|    3|0130|  8| 19| 80| 00| ff| fe| 7f| 07| 60|
|    4|0131|  8| 17| 80| 00| 00| 65| 7f| 07| 9f|
+-----+----+---+---+---+---+---+---+---+---+---+
only showing top 5 rows

